In [1]:
import gzip
from collections import defaultdict
from sklearn import linear_model
import csv
import pandas as pd
import os
import random
import numpy as np
import re
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression

In [3]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

In [3]:
#Task 1 & 2
fp = os.path.join("data","trainInteractions.csv")
data = pd.read_csv(fp)
training = data[0:400000]
validation = data[400000:]
#all unique recipe ids
all_recipes = list(data['recipe_id'].unique())

In [4]:
#taking the user_id, recipe_id, and cooked columns from validation set 
validation = validation.reset_index(drop = True)
validation = validation.drop(['date', 'rating'], axis =1)

In [5]:
#create negative sample for validation set 
neg_sample_pd = pd.DataFrame(columns = ['user_id','recipe_id',"cook_prediction"])
users = []
recipes = []
cook_predictions = []
for i in validation['user_id']:
    while True:
        random_recipe = random.choice(all_recipes)
        if random_recipe not in list(validation.loc[validation['user_id'] == i]['recipe_id']):
            users.append(i)
            recipes.append(random_recipe)
            cook_predictions.append(0)
            break
        
neg_sample_pd['user_id'] = users
neg_sample_pd['recipe_id'] = recipes
neg_sample_pd['cook_prediction'] = cook_predictions

In [6]:
#combine the positive validation set and the negative sample entry 
validation['cook_prediction'] = 1
combined_validation = pd.concat([validation, neg_sample_pd], ignore_index = True)

In [7]:
#Q1
recipeCount = defaultdict(int)
totalCooked = 0
''''''
for user,recipe,_ in readCSV(os.path.join('data',"trainInteractions.csv.gz")):
  recipeCount[recipe] += 1
  totalCooked += 1

mostPopular = [(recipeCount[x], x) for x in recipeCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(int(i))
  if count > totalCooked/2: break

test_id = []
test_recipe = []
test_prediction = []
'''
for l in open(os.path.join("data", "stub_Made.txt")):
  u,i = l.strip().split('-')
  if i in return1:
    predictions.write(u + '-' + i + ",1\n")
  else:
    predictions.write(u + '-' + i + ",0\n")
'''
for i in combined_validation.index:
    user = combined_validation['user_id'][i]
    recipe = combined_validation['recipe_id'][i]
    test_id.append(user)
    test_recipe.append(recipe)
    if recipe in return1:
        test_prediction.append(1)
    else:
        test_prediction.append(0)



In [8]:
#put predictions into a dataframe
predictions_combined_df = pd.DataFrame(columns = ['user_id','recipe_id','cook_prediction'])
predictions_combined_df['user_id'] = test_id
predictions_combined_df['recipe_id'] = test_recipe
predictions_combined_df['cook_prediction'] = test_prediction

In [9]:
#appending actual results to the dataframe for comparison
predictions_combined_df['actual_pred'] = combined_validation['cook_prediction']

In [10]:
#taking out the numerical columns for comparison
preds = predictions_combined_df[['actual_pred','cook_prediction']]

In [11]:
#Q1 Response
TP = preds[(preds['actual_pred'] == 1) & (preds['cook_prediction'] == 1)].shape[0]
TN = preds[(preds['actual_pred'] == 0) & (preds['cook_prediction'] == 0)].shape[0]
FP = preds[(preds['actual_pred'] == 0) & (preds['cook_prediction'] == 1)].shape[0]
FN = preds[(preds['actual_pred'] == 1) & (preds['cook_prediction'] == 0)].shape[0]
P = TP + FN
N = TN + FP
accuracy = ((TP + TN)/(P + N))*100
accuracy

69.4875

In [12]:
#QUESTION 2
def model(n):
    recipeCount = defaultdict(int)
    totalCooked = 0

    for user,recipe,_ in readCSV(os.path.join('data',"trainInteractions.csv.gz")):
      recipeCount[recipe] += 1
      totalCooked += 1

    mostPopular = [(recipeCount[x], x) for x in recipeCount]
    mostPopular.sort()
    mostPopular.reverse()

    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(int(i))
        if count > totalCooked/n: break

    test_id = []
    test_recipe = []
    test_prediction = []
    for i in combined_validation.index:
        user = combined_validation['user_id'][i]
        recipe = combined_validation['recipe_id'][i]
        test_id.append(user)
        test_recipe.append(recipe)
        if recipe in return1:
            test_prediction.append(1)
        else:
            test_prediction.append(0)
    
    predictions_combined_df = pd.DataFrame(columns = ['user_id','recipe_id','cook_prediction'])
    predictions_combined_df['user_id'] = test_id
    predictions_combined_df['recipe_id'] = test_recipe
    predictions_combined_df['cook_prediction'] = test_prediction
    return predictions_combined_df

def accuracy_func(preds):
    TP = preds[(preds['actual_pred'] == 1) & (preds['cook_prediction'] == 1)].shape[0]
    TN = preds[(preds['actual_pred'] == 0) & (preds['cook_prediction'] == 0)].shape[0]
    FP = preds[(preds['actual_pred'] == 0) & (preds['cook_prediction'] == 1)].shape[0]
    FN = preds[(preds['actual_pred'] == 1) & (preds['cook_prediction'] == 0)].shape[0]
    P = TP + FN
    N = TN + FP
    accuracy = ((TP + TN)/(P + N))*100
    return accuracy

results = []
for i in [10/9, 9/8, 8/7, 7/6, 6/5, 5/4, 4/3, 3/2, 2, 1, 1/2, 1/3, 1/6]:
    test = model(i)
    test['actual_pred'] = combined_validation['cook_prediction']
    test_comparison = test[['actual_pred','cook_prediction']]
    results.append(accuracy_func(test_comparison))

results
    

[61.4955,
 62.812999999999995,
 64.4055,
 66.4395,
 67.676,
 68.848,
 70.4805,
 71.36749999999999,
 69.4875,
 50.0,
 50.0,
 50.0,
 50.0]

In [13]:
#Question 3
#put users and recipes into dictionary 
usersPerRecipe = defaultdict(set)
recipesPerUser = defaultdict(set)

for i in list(training.index):
    user = training['user_id'][i]
    recipe = training['recipe_id'][i]
    usersPerRecipe[recipe].add(user)
    recipesPerUser[user].add(recipe)


def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

def jaccard_model(i,u,N):
    similarities = []
    users = usersPerRecipe[i]
    for i2 in recipesPerUser[u]:
        if i2 == i: continue
        sim = Jaccard(users, usersPerRecipe[i2])
        similarities.append(sim)
    if len(similarities) == 0:
        return 0
    if max(similarities) > N:
        return 1
    else:
        return 0

#compute jaccard for each user and their recipe in validation set 
jaccard_preds = []
for i in list(combined_validation.index):
    test_recipe = combined_validation['recipe_id'][i]
    test_user = combined_validation['user_id'][i]
    jaccard_preds.append(jaccard_model(test_recipe, test_user, 0.01))

combined_validation['jaccard_prediction'] = jaccard_preds

#compute accuracy
def accuracy_func2(preds):
    TP = preds[(preds['jaccard_prediction'] == 1) & (preds['cook_prediction'] == 1)].shape[0]
    TN = preds[(preds['jaccard_prediction'] == 0) & (preds['cook_prediction'] == 0)].shape[0]
    FP = preds[(preds['jaccard_prediction'] == 0) & (preds['cook_prediction'] == 1)].shape[0]
    FN = preds[(preds['jaccard_prediction'] == 1) & (preds['cook_prediction'] == 0)].shape[0]
    P = TP + FN
    N = TN + FP
    accuracy = ((TP + TN)/(P + N))*100
    return accuracy

accuracy_func2(combined_validation)

59.463

In [36]:
#Q4

#count recipes to determine popularity 
recipeCount = defaultdict(int)
totalCooked = 0

for user,recipe,_ in readCSV(os.path.join('data',"trainInteractions.csv.gz")):
  recipeCount[recipe] += 1
  totalCooked += 1

mostPopular = [(recipeCount[x], x) for x in recipeCount]
mostPopular.sort()
mostPopular.reverse()

mostPopularlist = []
for i in range(len(mostPopular)):
    mostPopularlist.append(int(mostPopular[i][1]))

def jaccard_popularity_model(i,u,N, l):
    similarities = []
    users = usersPerRecipe[i] 
    most_pop = mostPopularlist[:int(l*len(mostPopularlist))]
    for i2 in recipesPerUser[u]:
        if i2 == i: continue
        sim = Jaccard(users, usersPerRecipe[i2])
        similarities.append(sim)
    if len(similarities) == 0:
        return 0
    if i in most_pop:
            return 1
    if max(similarities) > N:
        return 1
    else:
        return 0

    
#compute jaccard & popularity prediction for each user and their recipe in validation set 
jaccard_popular_preds = []
for i in list(combined_validation.index):
    test_recipe = combined_validation['recipe_id'][i]
    test_user = combined_validation['user_id'][i]
    jaccard_popular_preds.append(jaccard_popularity_model(test_recipe,test_user,0.01, 0.25))

combined_validation['jaccard+pop_threshold_prediction'] = jaccard_popular_preds


#compute accuracy 
def accuracy_func3(preds):
    TP = preds[(preds['jaccard+pop_threshold_prediction'] == 1) & (preds['cook_prediction'] == 1)].shape[0]
    TN = preds[(preds['jaccard+pop_threshold_prediction'] == 0) & (preds['cook_prediction'] == 0)].shape[0]
    FP = preds[(preds['jaccard+pop_threshold_prediction'] == 0) & (preds['cook_prediction'] == 1)].shape[0]
    FN = preds[(preds['jaccard+pop_threshold_prediction'] == 1) & (preds['cook_prediction'] == 0)].shape[0]
    P = TP + FN
    N = TN + FP
    accuracy = ((TP + TN)/(P + N))*100
    return accuracy



accuracy_func3(combined_validation)

62.1845

In [60]:
#Q5 & Task3
predictions = open("predictions_Made.txt", 'w')
for l in open(os.path.join("data","stub_Made.txt")):
    if l.startswith("user_id"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    p = jaccard_popularity_model(int(i), int(u), 0.01, 0.25)
    if p == 1:
        predictions.write(u + '-' + i + ",1\n")
    else:
        predictions.write(u + '-' + i + ",0\n")

predictions.close()

#kaggle username is Jason Lee 002

In [4]:
#Q6
fp = os.path.join("data","trainRecipes.json.gz")
data = []

for d in readGz(fp):
    data.append(d)

training = data[0:190000]
validation = data[190000:]

wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in training:
    for w in d['steps'].split():
        w = ''.join([c for c in w.lower() if not c in punctuation])
        wordCount[w] += 1
        
words = pd.DataFrame()
words['word'] = list(wordCount.keys())
words['frequency'] = list(wordCount.values())
words = words.sort_values(by = ['frequency'], ascending = False).reset_index(drop = True).drop(index = 0).reset_index(drop = True).iloc[:1000]
words.iloc[:10]
        


,word,frequency
0,and,930718
1,the,866529
2,in,486904
3,a,473168
4,to,469609
5,with,313537
6,until,284197
7,add,259348
8,minutes,239941
9,of,235791


In [5]:
#Question 7
'''
top_1000 = defaultdict(int)
for i in list(words.index):
    top_1000[words['word'][i]] = 0

bag_of_words = pd.DataFrame()
bag_of_words['word_index'] = list(words['word'])
total_freq = []

for d in training:
    count = top_1000
    lst = list(top_1000.keys())
    for w in d['steps'].split():
        w = ''.join([c for c in w.lower() if not c in punctuation])
        if w in lst:
            count[w] += 1
    total_freq.append(list(pd.DataFrame(index = list(count.keys()),data = list(count.values()))[0]))
    #bag_of_words = pd.concat([bag_of_words, freq], axis = 1)

for i in range(len(total_freq)):
    out = pd.Series(total_freq[i])
    bag_of_words = pd.concat([bag_of_words, out], axis = 1)
'''
steps_lst = []
for d in training:
    step = d['steps']
    #step = re.sub(r'\s', ' ', step)
    steps_lst.append(step)
    
all_steps = pd.DataFrame(steps_lst, columns = ['steps'])
all_steps['steps'] = all_steps['steps'].apply(lambda x: re.sub("[^0-9a-zA-Z]+"," ", x).lower())

In [4]:
CountVec = CountVectorizer(ngram_range=(1,1), tokenizer = lambda x: x.split(), max_features = 1000)
Count_data = CountVec.fit_transform(all_steps['steps'])
cv_dataframe=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())

Y = []
for i in range(len(training)):
    Y.append(training[i]['minutes'])
    
reg = LinearRegression()
reg.fit(cv_dataframe,Y)

valid_steps = []
for d in validation:
    step = d['steps']
    #step = re.sub(r'\s', ' ', step)
    steps_lst.append(step)
all_steps_valid = pd.DataFrame(valid_steps, columns = ['steps'])
all_steps_valid['steps'] = all_steps_valid['steps'].apply(lambda x: re.sub("[^0-9a-zA-Z]+"," ", x).lower())
CountVec = CountVectorizer(ngram_range=(1,1), tokenizer = lambda x: x.split(), max_features = 1000)
Count_data = CountVec.fit_transform(all_steps_valid['steps'])
cv_dataframe2=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())

x_pred = reg.predict(cv_dataframe2)

Y_valid = []
for i in range(len(validation)):
    Y_valid.append(validation[i]['minutes'])

mse = np.square(np.subtract(Y_valid,x_pred)).mean()
mse

7503.729766534704

In [5]:
#Question 8
#changing max features to 95 instead of 1000
CountVec = CountVectorizer(ngram_range=(1,1), tokenizer = lambda x: x.split(), max_features = 95)
Count_data = CountVec.fit_transform(all_steps['steps'])
cv_dataframe=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())

Y = []
for i in range(len(training)):
    Y.append(training[i]['minutes'])

reg = LinearRegression()
reg.fit(cv_dataframe,Y)

ids = []
steps = []
for d in readGz(os.path.join("downloads/assignment1","testRecipes.json.gz")):
    ids.append(d['recipe_id'])
    steps.append(d['steps'])
    
res = pd.DataFrame()
res['steps'] = steps
res['steps'] = res['steps'].apply(lambda x: re.sub("[^0-9a-zA-Z]+"," ", x).lower())

Count_data = CountVec.fit_transform(res['steps'])
cv_dataframe=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())

x_pred = reg.predict(cv_dataframe)

res['recipe_id'] = ids
res['pred'] = x_pred

preds = open("prediction_Minutes.txt", 'w')
preds.write("recipe_id,prediction\n")
for i in range(0, res.shape[0]):
    preds.write(res['recipe_id'][i] + ',' + str(res['pred'][i]) + '\n')
preds.close()

#kaggle username is Jason Lee 002
